# Preparing Inputs for NGEN Modeling Framework

**Authors:**  
   - Tony Castronova <acastronova@cuahsi.org>    
   - Irene Garousi-Nejad <igarousi@cuahsi.org>  
    
**Last Updated:** 05.15.2023   

**Description**:  

The purpose of this Jupyter Notebook is to demonstrate the process of preparing inputs required to execute the [NOAA Next Generation (NextGen) Water Resource Modeling Framework](https://github.com/NOAA-OWP/ngen). These inputs consist of the following components: 

- Hydrologic and hydrodynamic graphs based on the National Hydrologic Geospatial (Hydrofabric) data which includes catchments, nexus, and flowlines. 
- Model domain parameters represented as configuration files.
- Meteorological forcing data.

 The Hydrofabric data can be accessed publicly through the AWS catalog. In this notebook, we use the **pre-release** version of the dataset, which represents the most recent version available on the Amazon S3 Bucket at the time of developing this notebook (https://nextgen-hydrofabric.s3.amazonaws.com/index.html#pre-release/). The configuration files encompass model default parameters, formulations, input and output paths, simulation time step, initial conditions, and other relevant settings. This example demonstrates the retrieval of hydrofabric data, followed by the extraction of necessary infromation for creating the parameter configuration file. These files are created for running Conceptual Functional Equivalent (CFE) model and Simple Logical Tautology Handler (SLoTH) in the NGEN framework. Note that the current version of this Jupyter notebook focuses on addressing the first two inputs, namely hydrofabric and configuration files. Future work will include steps to prepare the forcing dataset. 

 **Software Requirements**:  

The software and operating system versions used to develop this notebook are listed below. To avoid encountering issues related to version conflicts among Python packages, we recommend creating a new environment variable and installing the required packages specifically for this notebook.

Tested on: MacOS Ventura 13.2.1  

> boto3: 1.26.76  \  
> dask-core: 2023.4.0  \  
> fiona: 1.9.3  \  
> fsspec: 2023.4.0  \  
> geopandas: 0.12.2  \   
>  ipyleaflet: 0.17.2  \  
>  ipywidgets: 7.7.5  \   
>  matplotlib: 3.7.1  \   
>  netcdf4: 1.6.3  \   
>  numpy: 1.24.2  \  
>  pandas: 2.0.0  \  
>   requests: 2.28.2  \  
> s3fs: 2023.4.0  \  
> scipy: 1.10.1  \  
> xarray: 2023.4.1
  
**Supplementary Code**

This notebook relies on the following external scripts:  
- `subset.py` - A script originally written by Nels Frazier to subset the NGen Hydrofabric


---

In [5]:
import os
import time
import pandas
import subset
import pyproj
import datetime
import geopandas
import ipyleaflet
import geopandas as gpd
from pathlib import Path
from sidecar import Sidecar
from requests import Request
from ipywidgets import Layout

## 1. Create a map and load the Hydrofabric VPU geometries

The following cell creates an interactive map that encompasses the Hydrofabric VPU (Vector Processing Units) geometries. These geometries have been prepared ahead of time and are stored in a HydroShare [resource](https://www.hydroshare.org/resource/35e8c6023c154b6298fcda280beda849/). HydroShare offers convinient access to [WMS (Web Map Service)](https://docs.geoserver.org/latest/en/user/services/wms/index.html) and [WFS (Web Feature Service)](https://docs.geoserver.org/latest/en/user/services/wfs/index.html) capabilities, allowing us to easily display these geometries on an interactive map. These services facilitate the visualization and exploration of the data in a user-friendly manner.


In [6]:
# define parameters for creating the map
hs_wms_res = 'HS-e8ddee6a8a90484fa7a976458e79c0c3'
defaultLayout=Layout(width='960px', height='940px')
map_center = (41.74614949822607, -111.76617850993877) # logan, UT

# use ipyleaflet.Map to initialize a new instance of the `Map`
# class from ipyleaflet library and assing it to the variable m
m = ipyleaflet.Map(
    basemap=ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.OpenStreetMap.Mapnik, layout=defaultLayout),
    center=map_center,
    zoom=9,
    scroll_wheel_zoom=True,
    tap=False  
    )

# add USGS gages from the ArcGIS server that hosts the data
m.add_layer(
    ipyleaflet.WMSLayer(
        url='http://arcgis.cuahsi.org/arcgis/services/NHD/usgs_gages/MapServer/WmsServer',
        layers='0',
        transparent=True,
        format='image/png',
        min_zoom=8,
        max_zoom=18,
        )
)

# add the CONUS VPU boundaries
m.add_layer(
    ipyleaflet.WMSLayer(
        url=f'https://geoserver.hydroshare.org/geoserver/{hs_wms_res}/wms?',
        layers='vpu_boundaries',
        format='image/png',
        transparent=True,
        opacity=0.5,
        min_zoom=4,
        max_zoom=8
    )
)

# add the watershed VPU boundaries for each region.
for vpu in ['16']: 
    #['01', '02','03N','03S','03W', '04','05','06','07','08','09','10L','10U','11','12','13','14','15','16','17','18']:
    m.add_layer(
        ipyleaflet.WMSLayer(
            url=f'https://geoserver.hydroshare.org/geoserver/{hs_wms_res}/wms?',
            layers=f'{vpu}_boundaries',
            format='image/png',
            transparent=True,
            opacity=0.5,
            min_zoom=8,
            max_zoom=18
        )
    )


In [7]:
m

Map(center=[41.74614949822607, -111.76617850993877], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## 2. Enable interactive geometry selection 

Now, let's introduce an event handler that empowers us to highlight and store geometries that are clicked on the map. The following function utilizes the `WFS` endpoint to fetch the shape's boundary when a geometry is clicked on the map. This boundary will be drawn on the map, enabling us to visually identify the selected area.Additionally, we will store the information about the selected area for later use, particularly when we need to collect the upstream catchments associated with the selected area. 

In [12]:
# an empty dataframe to store the information of the selected region
selected_df = None

def handle_map_interaction(**kwargs):
    global selected_df
    
    if kwargs.get('type') == 'click':
        coords = kwargs['coordinates'] 
        url = f'https://geoserver.hydroshare.org/geoserver/{hs_wms_res}/wfs?' \
               'service=wfs&version=2.0.0&' \
              f'request=getFeature&' \
               'srsName=EPSG:4269&' \
              f'bbox={coords[1]},{coords[0]},{coords[1]},{coords[0]},EPSG:4269&' \
              f'typeName=vpu_boundaries&' \
               'outputFormat=json&' \
               'PropertyName=VPU'
        q = Request('GET', url).prepare().url
        df = gpd.read_file(q, format='json')
        
        # exit if a VPU is not found, i.e. a user doesn't click on the layer
        if len(df) == 0: return
        
        VPU = df.VPU.values[0]
        url = f'https://geoserver.hydroshare.org/geoserver/{hs_wms_res}/wfs?' \
               'service=wfs&version=2.0.0&' \
              f'request=getFeature&' \
               'srsName=EPSG:4269&' \
              f'bbox={coords[1]},{coords[0]},{coords[1]},{coords[0]},EPSG:4269&' \
              f'typeName={VPU}_boundaries&' \
               'outputFormat=json&'
        q = Request('GET', url).prepare().url
        df = gpd.read_file(q, format='json')

        # exit if a VPU is not found, i.e. a user doesn't click on the layer
        if len(df) == 0: return
    
        # save vpu region, convert crs, and save selection for later
        df['VPU'] = VPU 
        df = df.to_crs('EPSG:4269')
        selected_df = df
        
        if type(m.layers[-1]) == ipyleaflet.WKTLayer:
            m.remove_layer(m.layers[-1])
            
        # display the watershed boundary on the map
        m.add_layer(ipyleaflet.WKTLayer(wkt_string=df.iloc[0].geometry.wkt))
        
m.on_interaction(handle_map_interaction)

Display the map on side using `Sidecar` library.

In [13]:
sc = Sidecar(title='NGEN HydroFabric Map')
with sc:
    display(m)

Select the VPU of interest. Run the following code cell to print the `id` of the selected area. 

In [15]:
selected_df.id

0    wb-2917755
Name: id, dtype: object

## 3. Subset hydrofabric data for the selected area

The following code passes the `id` and `VPU` values of the geometries selected on the map (`selected_df`) to the hydrofabric subsetting script (`subset.py`). The subsetting algorithm implemented in the code adopts a reverse tracing technique called `subset_upstream`. It systematically identifies and selects all the upstream divides, catchments, nexuses, and flowlines starting from the most downstream nexus linked to the chosen geometries. 

In [16]:
# create lists of ids and vpus 
ids = list(selected_df.id.values)
vpus = list(selected_df.VPU.values)

output_files = []
for i in range(0, len(ids)):    
    print(50*'-'+f'\nProcessing VPU {vpus[i]}, {ids[i]} ')
    st = time.time()
    # build the hydrofabric_url
    hydrofabric_url = f's3://nextgen-hydrofabric/pre-release/nextgen_{vpus[i]}.gpkg'
    subset.subset_upstream(hydrofabric_url, ids[i])
    
    # move these files into a subdir to keep things orderly
    counter = 1
    outpath = ids[i]
    while os.path.exists(outpath):
        outpath = ids[i] + " (" + str(counter) + ")"
        counter += 1
    os.mkdir(outpath)
    for f in [f'{ids[i]}_upstream_subset.gpkg',
              'catchments.geojson',
              'crosswalk.json',
              'flowpath_edge_list.json',
              'flowpaths.geojson',
              'nexus.geojson',
              'cfe_noahowp_attributes.csv']:
        os.rename(f, f'{outpath}/{f}')
        
    # output_files.append(f'{ids[i]}_upstream_subset.gpkg')
    print(f'Output files located at: {outpath}')
    print(f'Completed in {time.time() - st} seconds\n'+50*'-')    

outdir = Path(outpath)

--------------------------------------------------
Processing VPU 16, wb-2917755 
s3://nextgen-hydrofabric/pre-release/nextgen_16.gpkg
Building Graph Network


ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Poi

KeyboardInterrupt: 

Exception ignored in: 'fiona._env.log_error'
Traceback (most recent call last):
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1559, in log
    self._log(level, msg, args, **kwargs)
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1634, in _log
    self.handle(record)
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1644, in handle
    self.callHandlers(record)
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1706, in callHandlers
    hdlr.handle(record)
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 978, in handle
    self.emit(record)
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1114, in emit
    self.flush()
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1094, in flush
 

KeyboardInterrupt: 

Exception ignored in: 'fiona._env.log_error'
Traceback (most recent call last):
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1559, in log
    self._log(level, msg, args, **kwargs)
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1634, in _log
    self.handle(record)
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1644, in handle
    self.callHandlers(record)
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1706, in callHandlers
    hdlr.handle(record)
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 978, in handle
    self.emit(record)
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1114, in emit
    self.flush()
  File "c:\Users\Irenegarousi\anaconda3\envs\geo-cartopy-ngen\Lib\logging\__init__.py", line 1094, in flush
 

## 4. Add the Subset Hydrofabric to the map

Execute the following code cell to add the subset catchments and rivers from the geopackage file  as visual overlays on the map. In this section, we begin by reading the shapefiles stored within the geopackage file. We then proceed to transform the projection system of these shapes into Web Mercator, the desired coordinate reference system for our map. Next, we create a WKTLayer for each catchment and river in the subset shapefiles. A WKTLayer enables us to represent the shape's geometry using a WKT string, which provides a concise description of its spatial properties.

In [ ]:
# read the shapes
catchments = geopandas.read_file(f'{outdir/outdir.name}_upstream_subset.gpkg',
                          layer='divides')
rivers = geopandas.read_file(f'{outdir/outdir.name}_upstream_subset.gpkg',
                          layer='flowpaths')

# define the target projection as EPSG:4269 - Web Mercator
# this is the default crs for leaflet
target_crs = pyproj.Proj('4269')

# transform the shapefile into EPSG:4269
catchments = catchments.to_crs(target_crs.crs)
rivers = rivers.to_crs(target_crs.crs)

# add the catchments to the map
for idx, shape in catchments.iterrows():
    wkt = ipyleaflet.WKTLayer(wkt_string=shape.geometry.wkt)
    wkt.style = {'color': 'green'}
    m.add_layer(wkt)
    
# add the rivers to the map
for idx, shape in rivers.iterrows():
    wkt = ipyleaflet.WKTLayer(wkt_string=shape.geometry.wkt)
    wkt.style = {'color': 'blue'}
    m.add_layer(wkt)

## 5. Create parameters configuration files